Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
from stockanalyzer import analyze_stock, stock_data_setup
import importlib

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

# Initialize an empty list to store the stocks
stocks_list = []

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

### Stocks Data

In [ ]:
analysis = analyze_stock(
	ticker= ['MSFT'], 
	dic_files=dic_files,
	dic_assistants=dic_assistants
)

In [5]:
print(analysis)

NameError: name 'analysis' is not defined

In [3]:
ticker = ['MRNA']

client = OpenAI()

file_stocks = stock_data_setup(client=client, ticker=ticker, type='price', dic_files=dic_files)
file_cashflow = stock_data_setup(client=client, ticker=ticker, type='cash', dic_files=dic_files)
file_income_stmt = stock_data_setup(client=client, ticker=ticker, type='income', dic_files=dic_files)

# Have to manually update the tool_resources because the file_id can change
dic_assistants['fin_analyst']['tool_resources'] = {
	'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
}

with open('config/assistants.json', 'w') as json_file:
	json.dump(dic_assistants, json_file)
	print(f'assistants.json file has been updated')

fin_analyst = AgentHandler(
	client = client, 
	new=False,
	assistant_name = 'fin_analyst',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

fin_consultant = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'fin_consultant',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

# prompt_start = f"This is your client. I want you to advice me on the stocks I list below.\
# 	I want to know the stocks' performance, if they're overvalued or undervalued, and whether or not they'll be a good investment.\
# 	I am looking for a long term (5+ years) investment and I have a relatively high risk tolerance.\
# 	Stocks: {ticker}"

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_stocks.csv, file id: file-Tefw5m8pdLpmdhiM1KsqI4rF has been deleted
file name: df_stocks.csv is uploaded, new file id: file-aGcHBmPdH14sXdEEipa89VXN
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_cashflow.csv, file id: file-I2tQB4Bkjyv9jEprAt1xUxG7 has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-McebaeiomNJMW1qsiq0yXHAu
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_income_stmt.csv, file id: file-WOPOtII6xsbhScFbnjenBnS9 has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-N7s7SaE2oanvYlYwZKU4IC8s
openai_upload_files/openai_files.json file has been updated
assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U


In [4]:
prompt_start = f"This is the financial consultant.\
	The client wants advice on the following stocks.\
	I need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\
	You will be provided with the data of these stocks in .csv format. Use them as you see fit.\
	Stocks: {ticker}"

thread = ThreadManager(
	client=client,
	prompt=prompt_start
)

thread.run_thread(
	assistant=fin_analyst
	# prompt=prompt_start,
	# attachments=[file_stocks.file_id, file_cashflow.file_id, file_income_stmt.file_id]
)

next_prompt = thread.last_message

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > code_interpreter

import pandas as pd

# Load the data from the CSV file
file_path = '/mnt/data/file-aGcHBmPdH14sXdEEipa89VXN'
stock_data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
stock_data.head()
assistant > Here are the information gathered from the financial analyst: 
These are the stocks the client wants advice on: 
- MRNA

The provided data includes daily stock prices for Moderna (MRNA), covering the following columns:
- Date
- Open
- High
- Low
- Close
- Volume
- Dividends
- Stock Splits
- Ticker

### Steps for Analysis:
1. **Historical Price Analysis:** Assess the price trends, volatility, and key support/resistance levels over a defined period.
2. **Volume Analysis:** Evaluate the trading volume to understand market interest and liquidity.
3. **Dividend and Split Analysis:** While MRNA currently shows no dividend or stock splits, we will confirm if this continues historically.
4. **Statistical Measures:** C

In [9]:
next_prompt = 'This message is for testing. Can you give a short sample response.'

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)

print(thread.last_message)

This message is for testing. Can you give a short sample response.


In [34]:
thread.df_messages
client.beta.threads.messages.list(thread_id=thread.thread_id).data
# messages_data[0]
# type(file_cashflow)

[Message(id='msg_zqvD0gadE9hib0XfNfbS57CC', assistant_id='asst_mPlBQ4ZF8YIF238DWqWRhV4U', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='### Recommendation for Moderna (MRNA):\n\n#### Key Points:\n- **Current Price:** \\$68.28\n- **Volatility:** High\n- **Growth Potential:** Strong in mRNA technology\n- **Dividends:** None\n\n#### Recommendation:\nMRNA is suitable for growth-oriented investors willing to accept high risk for potential high returns. It may not be ideal for conservative or income-seeking investors due to its volatility and lack of dividends.\n\nLet me know if you need further details!'), type='text')], created_at=1726350085, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_AFZhMbGJZSP5LrDRYHsMJDgD', status=None, thread_id='thread_uwelojSRusp6Zy4aWBAjYxwz'),
 Message(id='msg_6hilKxJmcKKogiyy5s4RaL8u', assistant_id=None, attachments=[], completed_at=None, content=

In [32]:
thread.messages

[{'message_id': 'msg_dy4lz80kcSMTj2CPUe5fyIXq',
  'assistant_id': None,
  'created_at': 1726350030,
  'file_ids': [],
  'role': 'user',
  'run_id': None,
  'message_text': "This is the financial consultant.\tThe client wants advice on the following stocks.\tI need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\tYou will be provided with the data of these stocks in .csv format. Use them as you see fit.\tStocks: ['MRNA']"},
 {'message_id': 'msg_zqvD0gadE9hib0XfNfbS57CC',
  'assistant_id': 'asst_mPlBQ4ZF8YIF238DWqWRhV4U',
  'created_at': 1726350085,
  'file_ids': [],
  'role': 'assistant',
  'run_id': 'run_AFZhMbGJZSP5LrDRYHsMJDgD',
  'message_text': '### Recommendation for Moderna (MRNA):\n\n#### Key Points:\n- **Current Price:** \\$68.28\n- **Volatility:** High\n- **Growth Potential:** Strong in mRNA technology\n- **Dividends:** None\n\n#### Recommendation:\nMRNA is suitable for growth-oriented investors willing to accept high risk for 

In [21]:
# existing_message_id = [dic['message_id'] for dic in thread.messages]

a = thread.messages[0]

thread.messages = []

thread.messages.append(a)

thread.get_last_message()
# thread.messages[-1]

get_last_message initiated
message_id: msg_zqvD0gadE9hib0XfNfbS57CC, assistant_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
asst_id = blank
new_message: ### Recommendation for Moderna (MRNA):

#### Key Points:
- **Current Price:** \$68.28
- **Volatility:** High
- **Growth Potential:** Strong in mRNA technology
- **Dividends:** None

#### Recommendation:
MRNA is suitable for growth-oriented investors willing to accept high risk for potential high returns. It may not be ideal for conservative or income-seeking investors due to its volatility and lack of dividends.

Let me know if you need further details!
message_id: msg_6hilKxJmcKKogiyy5s4RaL8u, assistant_id: None
message.assistant_id != asst_id, record previous message first
['### Recommendation for Moderna (MRNA):\n\n#### Key Points:\n- **Current Price:** \\$68.28\n- **Volatility:** High\n- **Growth Potential:** Strong in mRNA technology\n- **Dividends:** None\n\n#### Recommendation:\nMRNA is suitable for growth-oriented investors willing to acc

In [38]:
# existing_message_id = [dic['message_id'] for dic in thread.messages]
asst_id = ''

for message in messages_data:

	print(f'message_id: {message.id}, asst_id: {message.assistant_id}')

	if message.id in existing_message_id:
		print('id exists')

	else:
		# print('id doesnt exist')
		if asst_id == '':
			print('asst_id = blank')

			asst_id = message.assistant_id

		elif message.assistant_id == asst_id:

			print('message.assistant_id == asst_id, proceed normally')

		else:

			print('message.assistant_id != asst_id, start a new round of message')

message_id: msg_0YI1I3hF8OTzcEOG98L7Yi1V, asst_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
asst_id = blank
message_id: msg_QAPBtXXNgWnzX53d7SuW9dfQ, asst_id: None
message.assistant_id != asst_id, start a new round of message
message_id: msg_QVoKLLa1XshONulTZ3ZGpgwI, asst_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
message.assistant_id == asst_id, proceed normally
message_id: msg_H8VId2krDKV9oKD0G0ao59lF, asst_id: None
message.assistant_id != asst_id, start a new round of message
message_id: msg_Q5q5yM7L3n4Ng7F77GADFKON, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != asst_id, start a new round of message
message_id: msg_WJz18bE75m6GBxFfevGa0tgp, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != asst_id, start a new round of message
message_id: msg_itcdIzYvUErWcQaIbUeoNEAx, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != asst_id, start a new round of message
message_id: msg_uZbO2pgmoVS0p4i28XPKymwo, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != as

In [5]:
thread.delete_thread()

thread: thread_jq2tOUGEzC8Jp7jdePpkkgo7 has been deleted.
